# **Large-Scale Data Architecture Final Project**

As a travel agency in US, understanding flight traffic patterns and the impact of weather conditions is essential for anticipating changes in travel demand. Travel volume varies throughout the year due to seasonal weather, time periods, and passenger behavior, leading to clear peak and off-peak seasons. By analyzing historical air traffic passenger statistics for 5 major U.S. airports from 1999 to 2025, data on passenger counts alongside weather and time factors, this project aims to identify recurring travel patterns and predict upcoming peak seasons. These insights can help travel agencies prepare more effective support, promotions, and vouchers in advance, enabling better planning and more data-driven marketing strategies.

## Stakeholders & Value

The information from analyzing passenger numbers and weather data is very useful for different teams within the company:

**Marketing Department**:  
With this data, the marketing team can plan promotions and campaigns at the right time. They can offer vouchers or special deals when demand is high, making campaigns more effective and helping bring in more customers.

**Operation Manager**:
Operations managers can use the data to plan staff and resources better during busy periods. Knowing when passenger numbers are highest helps them organize check-ins, boarding, and other airport operations smoothly.

**Sales team**: The sales team can use this information to focus on high-demand periods and popular destinations. They can create better travel packages, identify opportunities for upselling, and plan offers that match customer needs, helping increase revenue.

## Pipeline Architecture

### Data Pipeline

![Alt text](Datapipeline.png)

**Data Collection**

The pipeline collects flight traffic data from CSV files and weather data from an external API. A Lambda function automatically triggers the API to fetch the latest weather information, ensuring the dataset is always up to date. Combining these datasets allows analysis of how weather impacts passenger traffic.

**Data Storage (S3)**

All raw data is stored in Amazon S3, providing scalable and cost-effective storage. S3 ensures durability and allows multiple teams to access the same data without creating duplicates. Both raw and processed data are kept for future analysis.

**Data Processing (Glue)**

AWS Glue cleans, transforms, and merges the flight and weather datasets into a structured format. Glue also catalogs the data, making it easy to query and ensuring consistency across the pipeline. This step prepares the data for analysis and reporting.

**Data Analysis (Athena)**

Amazon Athena allows querying the processed data directly from S3 using SQL without managing servers. Analysts can explore passenger trends, seasonal variations, and weather effects on flights. The insights support marketing, operations, customer service, and sales decisions.

### Cost of the system

- Amazon S3: Storing flight and weather data of about 10 GB per month costs roughly $0.23, with additional request costs of around $0.05. The total S3 cost is approximately $0.28 per month.

- AWS Lambda: Used to trigger the weather API. Since the number of requests is low and falls within the free tier, the Lambda cost is effectively $0 per month.

- AWS Glue: Handles ETL tasks such as cleaning and merging datasets. Assuming the ETL job runs for one hour each day using one DPU, the Glue cost is around $13.20 per month, which is the largest portion of the pipeline cost.

- Amazon Athena: Used to query processed data directly from S3. If about 50 GB of data is scanned per month, the cost is roughly $0.25 per month.

**Total Estimated Monthly Cost**: The overall monthly cost for the entire pipeline is approximately **$13.73**, with the majority coming from Glue ETL jobs.

### How these support the business goals
The designed data pipeline supports the travel agency’s business goals by providing valuable insights into passenger trends and weather patterns. It helps forecast peak travel periods, allowing the company to plan resources and operations efficiently. The pipeline also enables data-driven decisions for promotions, travel packages, and marketing strategies, which can increase revenue. By anticipating high-demand times and potential disruptions, it improves overall customer satisfaction. Additionally, the use of serverless services like S3, Lambda, Glue, and Athena keeps the pipeline cost-effective, as the company only pays for the resources it uses while maintaining scalability as data volumes grow.

## Key Performance Indicators

- **Monthly Passenger Volume Associated with Weather Conditions in Each Airport**: This KPI measures how passenger traffic varies at each airport in relation to weather conditions. It helps identify which airports are likely to be more crowded during certain months or under specific weather patterns. By understanding this, the travel agency can proactively prepare services such as taxi vouchers, airport transfers, and additional staff support, improving the customer experience and operational efficiency.

- **Average passenger volume per month**: Tracking the average number of passengers per month allows the agency to pinpoint peak travel seasons and periods of low demand. This information is crucial for planning marketing campaigns, promotions, and targeted offers. It also helps in resource planning, ensuring that operations and customer service teams are ready for busy periods while minimizing costs during off-peak months.

- **Yearly passenger volume per airlines**: This KPI shows which airlines are preferred by passengers over the years. It helps the agency understand customer loyalty and trends in airline selection. The insights can guide partnerships with airlines, special package deals, or targeted marketing strategies to attract or retain customers, ultimately supporting revenue growth and competitive positioning.

Collectively, these KPIs provide the agency with data-driven insights into passenger behavior, seasonal trends, airport usage, and airline preferences. This allows for better operational planning, marketing strategy, and customer service, ensuring the company can maximize efficiency, enhance customer satisfaction, and increase revenue

## Implementation

### Execution process

The execution begins with the creation of two S3 buckets: one to store flight traffic CSV files and another to store weather data retrieved from the Open Weather API. The weather data is collected using an AWS Lambda function, which retrieves weather information for the five major airports in the US. The Lambda function does not rely on manual API calls; instead, it is automated and serverless, allowing the system to fetch data on a scheduled basis or in response to an event. Once the Lambda function retrieves the weather data, it stores the results directly in the designated S3 bucket in a structured format, ready for further processing.

```
import json
import boto3
import requests
import os
from datetime import datetime

s3 = boto3.client("s3")
BUCKET = "weather-data-els9cy"

# 5 major airports
AIRPORTS = {
    "CA": {"lat": 35.35474, "lon": -116.885329},
    "AS": {"lat": 34.9428028, "lon": -97.8180194},
    "AZ": {"lat": 34.3055992, "lon": -112.165001},
    "AA": {"lat": 38.704022, "lon": -101.473911},
    "WN": {"lat": 46.25, "lon": -117.249001}
}

# Read API key from Lambda environment variable
API_KEY = os.environ.get("OPENWEATHER_API_KEY")

def lambda_handler(event, context):
    if not API_KEY:
        raise ValueError("OPENWEATHER_API environment variable is not set.")

    for airport, coords in AIRPORTS.items():
        data_list = []

        lat = coords["lat"]
        lon = coords["lon"]

        # OpenWeatherMap API URL for current weather
        url = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&units=metric&appid={API_KEY}"

        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()

            # Convert to the same structure as before
            record = {
                "year": datetime.utcfromtimestamp(data["dt"]).year,
                "month": datetime.utcfromtimestamp(data["dt"]).month,
                "coord": {"lat": lat, "lon": lon},
                "main": {
                    "temp": data["main"]["temp"],
                    "humidity": data["main"]["humidity"]
                },
                "weather": [
                    {
                        "main": data["weather"][0]["main"],
                        "description": data["weather"][0]["description"]
                    }
                ],
                "wind": {
                    "speed": data["wind"]["speed"],
                    "deg": data["wind"]["deg"]
                },
                "dt": data["dt"]
            }

            data_list.append(record)
        else:
            print(f"Failed to fetch data for {airport}: {response.status_code} {response.text}")

        # Upload JSON to S3
        s3_key = f"weather/{airport}.json"
        s3.put_object(Bucket=BUCKET, Key=s3_key, Body=json.dumps(data_list))
        print(f"Saved {airport} weather data to {s3_key}")

    return {
        "statusCode": 200,
        "body": "Weather data fetched from API and saved to S3."
    }
```

After creating two S3 buckets for the different types of datasets, the flight data is stored in CSV format, while the weather data retrieved from the API is stored in JSON format. Since JSON is semi-structured, it needs to be transformed into a column-oriented format to enable efficient querying later. To handle this, the first AWS Glue job was created, which cleans, parses, and converts the JSON weather data into a structured, query-ready format compatible with Athena.

```
#flight_weather_job
import sys
import boto3
import os
from awsglue.context import GlueContext
from pyspark.context import SparkContext
from pyspark.sql.functions import col, lit

# Initialize Glue/Spark
sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session

s3 = boto3.client('s3')

# Input JSON folder
input_path = "s3://weather-data-els9cy/weather/*.json"

# Temporary output folder
temp_output = "s3://weather-data-els9cy/temp_weather_parquet/"

# Final output file
final_output_bucket = "weather-data-els9cy"
final_output_key = "weather_parquet/weather_merged.parquet"

# Read JSON
weather_df = spark.read.option("multiline", "true").json(input_path)

# Flatten and cast numeric fields
flat_df = weather_df.select(
    col("year").cast("int"),
    col("month").cast("int"),
    col("coord.lat").cast("double").alias("lat"),
    col("coord.lon").cast("double").alias("lon"),
    col("main.temp").cast("double").alias("temp"),
    col("main.humidity").cast("double").alias("humidity"),
    col("weather")[0]["main"].alias("weather_main"),
    col("weather")[0]["description"].alias("weather_description"),
    col("wind.speed").cast("double").alias("wind_speed"),
    col("wind.deg").cast("double").alias("wind_deg"),
    col("dt").cast("bigint").alias("timestamp")
)

# Write to temporary folder as a single file
flat_df.coalesce(1).write.mode("overwrite").parquet(temp_output)

# Find the generated part-file and rename it
bucket_name = final_output_bucket
temp_prefix = "temp_weather_parquet/"
response = s3.list_objects_v2(Bucket=bucket_name, Prefix=temp_prefix)
for obj in response.get('Contents', []):
    key = obj['Key']
    if key.endswith(".parquet"):
        copy_source = {'Bucket': bucket_name, 'Key': key}
        s3.copy_object(Bucket=bucket_name, CopySource=copy_source, Key=final_output_key)
        s3.delete_object(Bucket=bucket_name, Key=key)
        break

# Delete the temporary folder (optional)
for obj in response.get('Contents', []):
    s3.delete_object(Bucket=bucket_name, Key=obj['Key'])

print(f"Parquet file saved as s3://{bucket_name}/{final_output_key}")
```
Five major airports weather data are saved seperately in the weather folder of S3. To make it easier to query, those five files should be merge by the following second ETL jobs

```
from awsglue.context import GlueContext
from pyspark.context import SparkContext
from pyspark.sql.functions import lit

# Initialize Glue/Spark
sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session

# Input folder containing the 5 Parquet files
input_path = "s3://weather-data-els9cy/weather_parquet/"

# Read all Parquet files in the folder
df = spark.read.parquet(input_path)

# Merge all data into 1 single file
output_path = "s3://weather-data-els9cy/weather_parquet_merged_single/"
df.coalesce(1).write.mode("overwrite").parquet(output_path)

print("All Parquet files merged into a single file successfully!")
```

After using AWS Glue ETL jobs to clean, transform, and merge the flight and weather datasets, an AWS Glue crawler is run to automatically scan the processed data stored in S3. The crawler detects the schema, identifies column names and data types, and creates or updates tables in the AWS Glue Data Catalog. This step ensures that the data is properly structured and ready for analysis. By automating the schema detection and cataloging process, the crawler reduces manual effort and ensures that the latest data is always accessible for reporting and business insights.

Two table were created in my database which are traffic flight data and weather data.

### Intepretation of the result

#### KPI 1: Monthly Passenger Volume Associated with Weather Conditions in Each Airport

```
WITH weather_mapped AS (
    SELECT *,
      CASE
        WHEN lat BETWEEN 35 AND 36 AND lon BETWEEN -117 AND -116 THEN 'CA'
        WHEN lat BETWEEN 34 AND 35 AND lon BETWEEN -98 AND -97 THEN 'AS'
        WHEN lat BETWEEN 34 AND 35 AND lon BETWEEN -113 AND -112 THEN 'AZ'
        WHEN lat BETWEEN 38 AND 39 AND lon BETWEEN -102 AND -101 THEN 'AA'
        WHEN lat BETWEEN 46 AND 47 AND lon BETWEEN -118 AND -117 THEN 'WN'
        ELSE 'UNKNOWN'
      END AS airport_code_visible
    FROM flight_db.weather_merged_v2
)

SELECT
    f."Operating Airline IATA Code" AS airport_code,

    CAST(f."Activity Period" / 100 AS INTEGER) AS year,
    CAST(f."Activity Period" % 100 AS INTEGER) AS month,

    SUM(f."Passenger Count") AS total_passengers,
    AVG(w.temp) AS avg_temp,
    AVG(w.wind_speed) AS avg_wind_speed,
    AVG(w.humidity) AS avg_humidity

FROM flight_db.raw f
JOIN weather_mapped w
  ON f."Operating Airline IATA Code" = w.airport_code_visible
 AND CAST(f."Activity Period" / 100 AS INTEGER) = w.year
 AND CAST(f."Activity Period" % 100 AS INTEGER) = w.month

GROUP BY
    f."Operating Airline IATA Code",
    CAST(f."Activity Period" / 100 AS INTEGER),
    CAST(f."Activity Period" % 100 AS INTEGER)

ORDER BY
    airport_code, year DESC, total_passengers DESC;
```

After executing the above query, I got the corresponding result:

![Alt text](KPI1.png)

The data for the example airport AA shows that passenger traffic varies throughout the year, with higher volumes during the summer months and lower volumes in the winter. This pattern corresponds closely with weather conditions, as months with higher average temperatures tend to have more passengers, while cooler months see fewer travelers. Wind speed and humidity show some variation but appear less influential than temperature. Understanding these trends allows the travel agency to anticipate which months the airport will be more crowded, enabling proactive planning of services such as taxi vouchers, airport transfers, and additional staff. By linking passenger volumes to weather patterns, the agency can improve operational efficiency and enhance the overall customer experience.

#### KPI 2: Average passenger volume per month

```
SELECT
    CAST(SUBSTRING("Activity Period Start Date", 6, 2) AS INTEGER) AS month,
    AVG("Passenger Count") AS avg_passengers
FROM "AwsDataCatalog"."flight_db"."raw"
GROUP BY CAST(SUBSTRING("Activity Period Start Date", 6, 2) AS INTEGER)
ORDER BY month;
```

![Alt text](KPI2.png)

The monthly passenger data reveals a clear seasonal pattern in travel demand. Passenger volumes peak in July and August, with averages of approximately 51,776 and 50,982, respectively, indicating the busiest travel period, likely corresponding to summer holidays. Moderate demand is observed in May and June, while the lowest passenger numbers occur in January and February, suggesting a slower winter period. From a business perspective, this information is critical for strategic planning. Marketing efforts and promotions can be targeted during low-demand months to stimulate travel, while premium services and seasonal packages can be emphasized during peak months to maximize revenue. Operational planning can also be optimized: staffing levels, fleet allocation, and customer service resources should be increased during high-demand periods to maintain service quality, while off-peak months provide opportunities for maintenance or cost-saving measures. Furthermore, revenue management strategies, such as dynamic pricing during peak periods and discounts during slow months, can help maximize profitability

#### KPI 3: Yearly passenger volume per airlines

```
SELECT 
    "Operating Airline" AS airline,
    SUBSTRING("Activity Period Start Date", 1, 4) AS year,
    SUM("Passenger Count") AS total_passengers
FROM "AwsDataCatalog"."flight_db"."raw"
GROUP BY "Operating Airline", SUBSTRING("Activity Period Start Date", 1, 4)
ORDER BY year DESC, total_passengers DESC;
```

![Alt text](KPI3.png)

The yearly passenger data highlights airline popularity and market trends. American Airlines and Alaska Airlines consistently lead in passenger volume, indicating strong customer preference and loyalty, while Southwest Airlines maintains moderate demand, and Air China and ITA Airways have limited market presence. For the agency, this insight can guide strategic decisions such as forming partnerships with high-demand airlines, creating targeted promotions or travel packages, and allocating marketing resources to attract or retain passengers. Leveraging these trends can improve revenue, enhance customer satisfaction, and strengthen competitive positioning in the airline market.
